In [1]:
import pandas as pd

data = pd.read_csv("train_qa.csv").reset_index(drop=True)

In [71]:
features = pd.read_csv('features_saved.csv')

In [72]:
features_1 = pd.read_csv('features_saved_1.csv')

In [73]:
features = features.drop(columns=['Unnamed: 0'])
features_1 = features_1.drop(columns=['Unnamed: 0'])


In [74]:
features = pd.concat([features, features_1]).reset_index(drop=True)

In [75]:
features.columns

Index(['paragraph_id', 'question_id', 'sent_prob', 'len_l', 'len_a', 'len_r',
       'cos_l', 'cos_a', 'cos_r', 'l_S', 'l_V', 'l_A', 'l_PR', 'l_ADV',
       'l_CONJ', 'm_S', 'm_V', 'm_A', 'm_PR', 'm_ADV', 'm_CONJ', 'r_S', 'r_V',
       'r_A', 'r_PR', 'r_ADV', 'r_CONJ', 'result'],
      dtype='object')

In [31]:
features = features.drop(columns=['Unnamed: 0'])

In [76]:
data[data['question_id']==2390].iloc[0]['question']

'Что можно получить при институте за новые задумки?'

In [77]:
sum(features['result'])

1653.0

In [78]:
from pymystem3 import Mystem

m = Mystem()
#    texts = list(map(lambda line: m.lemmatize(line.strip()), f.readlines()))

print(m.analyze("Что ты делаешь? Когда ты будешь? В каком году это было?"))

[{'analysis': [{'lex': 'что', 'wt': 0.2934446278, 'gr': 'SPRO,ед,сред,неод=(вин|им)'}], 'text': 'Что'}, {'text': ' '}, {'analysis': [{'lex': 'ты', 'wt': 1, 'gr': 'SPRO,ед,2-л=им'}], 'text': 'ты'}, {'text': ' '}, {'analysis': [{'lex': 'делать', 'wt': 1, 'gr': 'V,несов,пе=непрош,ед,изъяв,2-л'}], 'text': 'делаешь'}, {'text': '? '}, {'analysis': [{'lex': 'когда', 'wt': 0.8251525963, 'gr': 'CONJ='}], 'text': 'Когда'}, {'text': ' '}, {'analysis': [{'lex': 'ты', 'wt': 1, 'gr': 'SPRO,ед,2-л=им'}], 'text': 'ты'}, {'text': ' '}, {'analysis': [{'lex': 'быть', 'wt': 1, 'gr': 'V,нп=непрош,ед,изъяв,2-л'}], 'text': 'будешь'}, {'text': '? '}, {'analysis': [{'lex': 'в', 'wt': 0.9999917878000001, 'gr': 'PR='}], 'text': 'В'}, {'text': ' '}, {'analysis': [{'lex': 'какой', 'wt': 1, 'gr': 'APRO=(пр,ед,муж|пр,ед,сред)'}], 'text': 'каком'}, {'text': ' '}, {'analysis': [{'lex': 'год', 'wt': 1, 'gr': 'S,муж,неод=(дат,ед|местн,ед)'}], 'text': 'году'}, {'text': ' '}, {'analysis': [{'lex': 'это', 'wt': 0.780983373

In [79]:
def get_stat_mystem(phrase):
    analyzed = m.analyze(phrase)
    stats = {'S': 0, 'V': 0, 'A':0, 'PR':0, 'ADV': 0, 'CONJ': 0}
    for t in analyzed:
        if 'analysis' in t and len(t['analysis']) > 0:
            if 'gr' in t['analysis'][0]:
                gr = t['analysis'][0]['gr'].split('=')[0].split(',')[0]
                if gr in stats:
                    stats[gr] += 1
    l = len(analyzed)
    if l == 0:
        l = 1
    return list(map(lambda x: x / l, list(stats.values())))

In [80]:
def get_question_features(question):
    features = [*get_stat_mystem(question)]
    return features

In [81]:
question_features = pd.DataFrame(columns=['question_id', 'q_S', 'q_V', 'q_A', 'q_PR', 'q_ADV', 'q_CONJ',])

In [82]:
import numpy as np

question_ids = np.unique(features['question_id'])

In [83]:
for i, question_id in enumerate(question_ids):
    question = data[data['question_id']==int(question_id)].iloc[0]['question']
    q_features = get_question_features(question)
    question_features.loc[i] = [question_id, *q_features]

In [84]:
question_features

,question_id,q_S,q_V,q_A,q_PR,q_ADV,q_CONJ
0,84.0,0.210526,0.052632,0.000000,0.105263,0.000000,0.052632
1,199.0,0.176471,0.000000,0.117647,0.117647,0.000000,0.000000
2,290.0,0.204545,0.045455,0.090909,0.068182,0.000000,0.000000
3,424.0,0.115385,0.038462,0.192308,0.038462,0.000000,0.038462
4,500.0,0.190476,0.095238,0.047619,0.047619,0.000000,0.000000
5,541.0,0.151515,0.090909,0.030303,0.060606,0.000000,0.030303
6,586.0,0.157895,0.105263,0.052632,0.052632,0.000000,0.052632
7,608.0,0.200000,0.066667,0.066667,0.066667,0.000000,0.000000
8,614.0,0.272727,0.090909,0.000000,0.000000,0.000000,0.000000
9,675.0,0.173913,0.086957,0.043478,0.130435,0.000000,0.000000


In [85]:
temp = pd.merge(features, question_features, on=['question_id'])

In [86]:
temp

,paragraph_id,question_id,sent_prob,len_l,len_a,len_r,cos_l,cos_a,cos_r,l_S,...,r_PR,r_ADV,r_CONJ,result,q_S,q_V,q_A,q_PR,q_ADV,q_CONJ
0,14754.0,60544.0,0.715019,7.0,3.0,1.0,0.888011,0.000000,0.000000,0.214286,...,0.000000,0.000000,0.000000,1.0,0.176471,0.058824,0.058824,0.058824,0.000000,0.000000
1,14754.0,60544.0,0.000000,18.0,8.0,8.0,0.000000,0.000000,0.000000,0.200000,...,0.000000,0.000000,0.142857,0.0,0.176471,0.058824,0.058824,0.058824,0.000000,0.000000
2,14754.0,60544.0,0.057695,0.0,10.0,13.0,0.000000,0.000000,0.087499,0.000000,...,0.000000,0.000000,0.047619,0.0,0.176471,0.058824,0.058824,0.058824,0.000000,0.000000
3,14754.0,60544.0,0.057695,1.0,1.0,21.0,0.000000,0.000000,0.065450,0.500000,...,0.025641,0.000000,0.025641,0.0,0.176471,0.058824,0.058824,0.058824,0.000000,0.000000
4,14754.0,60544.0,0.057695,1.0,10.0,12.0,0.000000,0.000000,0.087499,0.500000,...,0.000000,0.000000,0.050000,0.0,0.176471,0.058824,0.058824,0.058824,0.000000,0.000000
5,14754.0,60544.0,0.057695,7.0,6.0,10.0,0.000000,0.000000,0.101881,0.133333,...,0.000000,0.000000,0.058824,0.0,0.176471,0.058824,0.058824,0.058824,0.000000,0.000000
6,14754.0,60544.0,0.057695,8.0,1.0,14.0,0.000000,0.000000,0.087499,0.117647,...,0.000000,0.000000,0.043478,0.0,0.176471,0.058824,0.058824,0.058824,0.000000,0.000000
7,14754.0,60544.0,0.057695,11.0,2.0,10.0,0.000000,0.000000,0.101881,0.173913,...,0.000000,0.000000,0.058824,0.0,0.176471,0.058824,0.058824,0.058824,0.000000,0.000000
8,13859.0,1604.0,0.083776,23.0,1.0,8.0,0.097260,0.000000,0.059390,0.225000,...,0.062500,0.000000,0.062500,1.0,0.222222,0.066667,0.044444,0.088889,0.000000,0.000000
9,13859.0,1604.0,0.083776,2.0,8.0,22.0,0.000000,0.000000,0.134695,0.000000,...,0.047619,0.023810,0.023810,0.0,0.222222,0.066667,0.044444,0.088889,0.000000,0.000000


In [87]:
sum(temp['q_ADV'])

112.05726874098161

In [88]:
temp.columns

Index(['paragraph_id', 'question_id', 'sent_prob', 'len_l', 'len_a', 'len_r',
       'cos_l', 'cos_a', 'cos_r', 'l_S', 'l_V', 'l_A', 'l_PR', 'l_ADV',
       'l_CONJ', 'm_S', 'm_V', 'm_A', 'm_PR', 'm_ADV', 'm_CONJ', 'r_S', 'r_V',
       'r_A', 'r_PR', 'r_ADV', 'r_CONJ', 'result', 'q_S', 'q_V', 'q_A', 'q_PR',
       'q_ADV', 'q_CONJ'],
      dtype='object')

In [89]:
right_order = ['paragraph_id', 'question_id', 'sent_prob', 'len_l', 'len_a', 'len_r',
       'cos_l', 'cos_a', 'cos_r', 'l_S', 'l_V', 'l_A', 'l_PR', 'l_ADV',
       'l_CONJ', 'm_S', 'm_V', 'm_A', 'm_PR', 'm_ADV', 'm_CONJ', 'r_S', 'r_V',
       'r_A', 'r_PR', 'r_ADV', 'r_CONJ', 'q_S', 'q_V', 'q_A', 'q_PR',
       'q_ADV', 'q_CONJ', 'result']

In [90]:
temp = temp[right_order]

In [91]:
temp.to_csv("features_wquests_saved.csv", sep=',')

In [92]:
len(right_order)

34

In [93]:
train_data = temp[right_order[:-1]].values

In [94]:
train_y = temp[right_order[-1]].values

In [130]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=8, n_estimators=100)

In [131]:
regr.fit(train_data, train_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=8,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [132]:
regr.score(train_data, train_y)

0.547658423896364

In [109]:
def gen_features_test(sent_variants, question, paragraph_id, question_id):
    data_x = []
    data_y = []
    variants = []
    for sent, prob in sent_variants:
        words = nltk.word_tokenize(sent, language='russian')
        for i in range(len(words)):
            for l in range(1, min(11, len(words) - i)):
                variant = words[i:(i + l)]
                tfidf = vectorizer.fit_transform([question, " ".join(words[:i]), " ".join(words[i:(i + l)]), " ".join(words[(i + l):])])
                cosine = (tfidf*tfidf.T).toarray()
                features = [paragraph_id, question_id, prob,
                    i, l, len(words) - i - l, 
                    cosine[0][1], cosine[0][2], cosine[0][3],
                    *get_stat_mystem(" ".join(words[:i])),
                    *get_stat_mystem(" ".join(words[i:(i + l)])),
                    *get_stat_mystem(" ".join(words[(i + l):])),
                    *get_question_features(question)]
                data_x.append(features)
                variants.append(variant)
    return data_x, variants

In [103]:
import nltk


nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/anna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

def gen_sent_variants(text, question):
    sents = nltk.sent_tokenize(text, language="russian")
    word_quests = nltk.word_tokenize(question, language='russian')
    all_sents = [" ".join(word_quests)]

    for sent in sents:
        words = nltk.word_tokenize(sent, language='russian')
        all_sents.append(" ".join(words))
    tfidf = vectorizer.fit_transform(all_sents)
    cosine = (tfidf * tfidf.T).toarray()
    result = list(range(1, len(all_sents)))
    result = map(lambda t: (sents[t - 1], cosine[0][t]), sorted(result, key=lambda t: cosine[0][t])[-3:])

    return list(result)

In [133]:
test_data = pd.read_csv("test_qa_6.csv", sep='\t').reset_index(drop=True)

In [134]:
with open('output_6', 'w') as fout:
    for i in range(len(test_data)):
        sent_vars = [gen_sent_variants(test_data['paragraph'][i], test_data['question'][i])[-1]]
        data_x, variants = gen_features_test(sent_vars, test_data['question'][i], test_data['paragraph_id'][i], test_data['question_id'][i])
        #print(len(data_x[0]))
        predicts = regr.predict(data_x)
        best = np.argmax(predicts)
        if predicts[best] < 0.25 or len(" ".join(variants[best])) < 2:
            print('{}\t{}'.format(test_data['question_id'][i], sent_vars[0][0]), file=fout, flush=True)
            continue
        print('{}\t{}'.format(test_data['question_id'][i], " ".join(variants[best])), file=fout, flush=True)

In [129]:
sorted(list(zip(regr.feature_importances_, right_order[:-1])), key=lambda x: x[0])[-10:]

[(0.019205872920351585, 'l_V'),
 (0.02726805785078306, 'r_S'),
 (0.03114069741184497, 'cos_r'),
 (0.05935842492460279, 'cos_l'),
 (0.06082592660997265, 'len_l'),
 (0.06889446785227538, 'cos_a'),
 (0.06988191220397427, 'len_a'),
 (0.08170566035159905, 'len_r'),
 (0.13562518026403067, 'm_V'),
 (0.22048207313136445, 'sent_prob')]

In [115]:
from sklearn import linear_model
regr_2 = linear_model.Ridge(alpha=.5)

In [116]:
regr_2.fit(train_data, train_y)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [117]:
regr_2.score(train_data, train_y)

0.27493623812924795